# <span style="color:green"><center>Diplomado en Big Data</center></span>

# <span style="color:red"><center>Ejecución con un cluster en la nube: Coiled<center></span>

<img src="../images/Coiled_Logo.jpg" align="right" width="30%">

##   <span style="color:blue">Profesores</span>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 

##   <span style="color:blue">Asesora Medios y Marketing digital</span>
 

4. Maria del Pilar Montenegro, pmontenegro88@gmail.com 

## <span style="color:blue">Contenido</span>


* [Introducción](#Introducción)
* [Lanzar un cluster](#Lanzar-un-cluster)
* [Analizar datos en la nube](#Analizar-datos-en-la-nube)
* [Detener un cluster](#Detener-un-cluster)
* [Entornos de software](#Entornos-de-software)
* [Crear un entorno de software](#Crear-un-entorno-de-software)


## <span style="color:blue">Fuente</span>

Esta es una traducción libre del tutorial disponible en [coiled-tutorial](https://cloud.coiled.io/examples/notebooks).

## <span style="color:blue">Introducción</span>

### Instalación de Coiled

## <span style="color:blue">Lanzar un cluster</span>

El primer paso es hacer girar un Dask Cluster. En Coiled, esto se hace creando una instancia `coiled.Cluster`, hay [ argumentos clave](https://docs.coiled.io/user_guide/api.html#coiled.Cluster) que puede utilizar para especificar más los detalles de su clúster. Por favor lea laa [cluster creation documentation](https://docs.coiled.io/user_guide/cluster_creation.html) para saber más.

Tenga en cuenta que le daremos un nombre a este clúster, si no especifica este argumento de palabra clave, los clústeres recibirán un nombre único generado aleatoriamente.

In [1]:
import coiled

cluster = coiled.Cluster(name="quickstart-example", n_workers=10)

/home/alvaro/anaconda3/envs/coiled/lib/python3.8/site-packages/rich/live.py:227: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')


Found software environment build


Una vez que se ha creado un clúster (puede ver el estado en su [Panel de control en espiral] (https://cloud.coiled.io/)), puede conectar Dask al clúster creando una instancia de *cliente distribuido*.

In [ ]:
from dask.distributed import Client

client = Client(cluster)
client

## <span style="color:blue">Analizar datos en la nube</span>

Ahora que tenemos nuestro clúster en ejecución y Dask conectado a él, ejecutemos un cálculo. Este ejemplo ejecutará el cálculo en aproximadamente 84 millones de filas.

In [ ]:
import dask.dataframe as dd

df = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-*.csv",
    dtype={
        "payment_type": "UInt8",
        "VendorID": "UInt8",
        "passenger_count": "UInt8",
        "RatecodeID": "UInt8",
    },
    storage_options={"anon": True},
    blocksize="16 MiB",
).persist()

df.groupby("passenger_count").tip_amount.mean().compute()

## <span style="color:blue">Detener un cluster</span>

De forma predeterminada, los clústeres se cerrarán después de 20 minutos de inactividad. Puede detener un clúster presionando el botón de parada en el [Coiled dashboard](https://cloud.coiled.io/). Alternativamente, podemos obtener una lista de todos los clústeres en ejecución y usar el nombre del clúster para detenerlo.

In [3]:
coiled.list_clusters()

{'quickstart-example': {'id': 18813,
  'status': 'running',
  'account': 'alvaro-montenegro',
  'dashboard_address': 'http://ec2-100-26-244-131.compute-1.amazonaws.com:8787',
  'configuration': 310,
  'options': {'region': 'us-east-1',
   'credentials': {},
   'account_role': None,
   'target_environment': 'dev',
   'spot': True},
  'address': 'tls://ec2-100-26-244-131.compute-1.amazonaws.com:8786'}}

El comando `list_clusters` devuelve un diccionario con el nombre del clúster utilizado como clave. Podemos tomar eso y luego llamar al comando `coiled.delete_cluster()` para detener el clúster en ejecución, y `client.close()` para cerrar el cliente.

In [4]:
coiled.delete_cluster(name="quickstart-example")
client.close()

Cluster deleted successfully.

Ahora puede regresa a [Coiled dashboard](https://cloud.coiled.io/) y verificar que el cluster está deteniendose/detenido

## <span style="color:blue">Entornos de software</span>


Los entornos de software son imágenes de Docker (el equivalente de Kubernet de Google) que contienen todas sus dependencias y archivos que podría necesitar para ejecutar sus cálculos. Si no especifica un entorno de software para el constructor `coiled.Cluster`, usaremos el entorno de software predeterminado de Coiled. Puede obtener más información sobre los entornos de software en nuestro[documentation](https://docs.coiled.io/user_guide/software_environment.html).

## <span style="color:blue">Crear un entorno de software</span>

Al crear entornos de software, hay [varios argumentos clave](https://docs.coiled.io/user_guide/api.html#coiled.create_software_environment) que puede utilizar para crear un entorno personalizado para su trabajo.

In [ ]:
coiled.create_software_environment(
    name="quickstart", 
    conda={
        "channels": ["conda-forge"], 
        "dependencies": ["coiled"]
    }
)

Ahora podemos seguir nuestro flujo de trabajo anterior de creación de un clúster; esta vez, usaremos nuestro entorno de software recién creado, conectar el clúster a Dask y luego ejecutar el mismo ejemplo.

In [ ]:
cluster = coiled.Cluster(n_workers=10, software="quickstart")
client = Client(cluster)
client

Si usted va a [Coiled dashboard](https://cloud.coiled.io/), en la columna **Entorno de software**, puede ver que estamos usando el entorno de software de inicio rápido que acabamos de crear. Tenga en cuenta también que esta vez, el clúster tendrá un nombre generado aleatoriamente.

Ahora ejecutemos el mismo cálculo que antes, pero utilizando el clúster que se ejecuta con el entorno de software que hemos creado recientemente.

In [ ]:
df = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-*.csv",
    dtype={
        "payment_type": "UInt8",
        "VendorID": "UInt8",
        "passenger_count": "UInt8",
        "RatecodeID": "UInt8",
    },
    storage_options={"anon": True},
    blocksize="16 MiB",
).persist()

df.groupby("passenger_count").tip_amount.mean().compute()